In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
os.environ['KAGGLE.CONFIG.DIR']="."
#Configuro el directorio porque en mi computadora no encontraba el archivo kaggle.json

In [3]:
from google.colab import files
!mkdir -p ~/.config/kaggle
!mv kaggle.json ~/.config/kaggle/
!chmod 600 ~/.config/kaggle/kaggle.json

In [4]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 731MB/s]


In [5]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [26]:
entrenamiento=pd.read_csv("train.csv")
testeo=pd.read_csv("test.csv")

In [40]:
entrenamiento.F_EDUCACIONPADRE.isna().sum()

np.int64(32137)

In [27]:
#Usando LabelEncoder para transformar los valores ordinales del rendimiento a valores numericos.
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
entrenamiento['RENDIMIENTO_NUM'] = le.fit_transform(entrenamiento['RENDIMIENTO_GLOBAL'])

#Escojo las columnas de interés para entrenar el modelo
columnas_seleccionadas = [
    'F_EDUCACIONPADRE',
    'F_EDUCACIONMADRE',
    'F_ESTRATOVIVIENDA',
    'F_TIENEINTERNET',
    'E_HORASSEMANATRABAJA'
]

In [47]:
def imputar_todas_las_columnas(entrenamiento, testing, columnas, columna_grupo='F_ESTRATOVIVIENDA'): #Voy a sacar la moda de cada columna por estrato

    if columna_grupo in entrenamiento.columns and entrenamiento[columna_grupo].isnull().any():
        moda_global = entrenamiento[columna_grupo].mode()
        if not moda_global.empty:
            entrenamiento[columna_grupo] = entrenamiento[columna_grupo].fillna(moda_global[0])
            testing[columna_grupo] = testing[columna_grupo].fillna(moda_global[0])

    for columna in columnas:
        if columna == columna_grupo:
            continue

        if columna in entrenamiento.columns:
            entrenamiento[columna] = entrenamiento.groupby(columna_grupo)[columna].transform(
                lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x.fillna(entrenamiento[columna].mode()[0])
            )

            if entrenamiento[columna].isnull().any():
                moda_global = entrenamiento[columna].mode()
                if not moda_global.empty:
                    entrenamiento[columna] = entrenamiento[columna].fillna(moda_global[0])

            testing[columna] = testing.groupby(columna_grupo)[columna].transform(
                lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x.fillna(entrenamiento[columna].mode()[0])
            )

            if testing[columna].isnull().any():
                testing[columna] = testing[columna].fillna(entrenamiento[columna].mode()[0])


    return entrenamiento, testing

entrenamiento, testeo = imputar_todas_las_columnas(entrenamiento, testeo, columnas_seleccionadas)

In [48]:
faltantes_tr = entrenamiento[columnas_seleccionadas].isnull().sum()
faltantes_ts = testeo[columnas_seleccionadas].isnull().sum()
print(faltantes_tr)
print(faltantes_ts)


F_EDUCACIONPADRE        0
F_EDUCACIONMADRE        0
F_ESTRATOVIVIENDA       0
F_TIENEINTERNET         0
E_HORASSEMANATRABAJA    0
dtype: int64
F_EDUCACIONPADRE        0
F_EDUCACIONMADRE        0
F_ESTRATOVIVIENDA       0
F_TIENEINTERNET         0
E_HORASSEMANATRABAJA    0
dtype: int64


In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Definir preprocessor, esto es para transformar las columnas númericas en escalares estándar y las columnas categóricas en one-hot
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), columnas_seleccionadas)
    ])
X_tr_procesado = preprocessor.fit_transform(entrenamiento[columnas_seleccionadas])
X_ts_procesado = preprocessor.transform(testeo[columnas_seleccionadas])

y_tr = entrenamiento['RENDIMIENTO_NUM']  # Usar la versión numérica codificada
print(f"Shapes: X_tr {X_tr_procesado.shape}, y_tr {y_tr.shape}, X_ts {X_ts_procesado.shape}")

Shapes: X_tr (692500, 33), y_tr (692500,), X_ts (296786, 33)


In [57]:
from sklearn.ensemble import RandomForestClassifier

rf_model =RandomForestClassifier(
    n_estimators=93,
    max_depth=17,
    min_samples_split=35,
    min_samples_leaf=22,
    n_jobs=-1,
    random_state=42
)

rf_model.fit(X_tr_procesado, y_tr)
predic=rf_model.predict(X_ts_procesado)

In [58]:
y_pred_original = le.inverse_transform(predic)
submission = pd.DataFrame([testeo.ID, pd.Series(y_pred_original, name="RENDIMIENTO_GLOBAL")]).T
submission.shape

(296786, 2)

In [59]:
submission.to_csv("Kaggle_Icfes_Competition_3.csv", index=False)

In [60]:
!head Kaggle_Icfes_Competition_3.csv

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,medio-alto
499179,alto
782980,bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,medio-bajo
